# **FAKE NEWS DETECTION**

Let us first load the libraries:

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt

import re
import string

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

Now, we import the dataset:

In [2]:
data_fake = pd.read_csv("../input/fake-news-detection-dataset/Fake.csv")
data_true = pd.read_csv("../input/fake-news-detection-dataset/True.csv")

In [3]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
# Inserting a column "class" as target feature
data_fake["class"] = 0
data_true["class"] = 1

In [5]:
data_fake.shape

(23481, 5)

In [6]:
data_true.shape

(21417, 5)

Let us remove last 10 columns for manual testing

In [7]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480,23470,-1):
    data_fake.drop([i], axis = 0, inplace = True)
    
    
data_true_manual_testing = data_true.tail(10)
for i in range(21416,21406,-1):
    data_true.drop([i], axis = 0, inplace = True)

In [8]:
data_fake.shape, data_true.shape

((23471, 5), (21407, 5))

Noice

In [9]:
data_fake_manual_testing["class"] = 0
data_true_manual_testing["class"] = 1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
data_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [11]:
 data_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [12]:
data_manual_testing = pd.concat([data_fake_manual_testing,data_true_manual_testing], axis = 0)
data_manual_testing.to_csv("manual_testing.csv")

In [13]:
# Merging Fake and True Dataframes
data_merge = pd.concat([data_fake, data_true], axis =0 )
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [14]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [15]:
# Removing columns which are not required
data = data_merge.drop(["title", "subject","date"], axis = 1)

In [16]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [17]:
# Random Shuffling the dataframe
data = data.sample(frac = 1)

In [18]:
data.head()

,text,class
111,WASHINGTON (Reuters) - The Republican-controll...,1
14243,George Soros seems to be sticking his nose int...,0
17258,Just when you thought the 2014 election result...,0
1131,WASHINGTON (Reuters) - Republican U.S. Senator...,1
10931,WASHINGTON (Reuters) - U.S. President Barack O...,1


In [19]:
data.reset_index(inplace = True)
data.drop(["index"], axis = 1, inplace = True)

In [20]:
data.columns

Index(['text', 'class'], dtype='object')

In [21]:
data.head()

,text,class
0,WASHINGTON (Reuters) - The Republican-controll...,1
1,George Soros seems to be sticking his nose int...,0
2,Just when you thought the 2014 election result...,0
3,WASHINGTON (Reuters) - Republican U.S. Senator...,1
4,WASHINGTON (Reuters) - U.S. President Barack O...,1


In [22]:
# Creating a function to process the texts
def word_opt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [23]:
data["text"] = data["text"].apply(word_opt)

In [24]:
# Defining dependent and independent variables
x = data["text"]
y = data["class"]

In [25]:
# Separating Training and Testing Data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [26]:
# converting text to vectors
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

# Logistic Regression

In [27]:
from sklearn.linear_model import LogisticRegression

LogReg = LogisticRegression()
LogReg.fit(xv_train,y_train)

LogisticRegression()

In [28]:
pred_lr = LogReg.predict(xv_test)

In [29]:
LogReg.score(xv_test, y_test)

0.9878787878787879

In [30]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5885
           1       0.99      0.99      0.99      5335

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



# Decision Tree Classification

In [31]:
from sklearn.tree import DecisionTreeClassifier

DecTree = DecisionTreeClassifier()
DecTree.fit(xv_train, y_train)

DecisionTreeClassifier()

In [32]:
pred_dectree = DecTree.predict(xv_test)

In [33]:
DecTree.score(xv_test, y_test)

0.996078431372549

In [34]:
print(classification_report(y_test, pred_dectree))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5885
           1       1.00      1.00      1.00      5335

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



# Gradient Boosting Classifier

In [35]:
from sklearn.ensemble import GradientBoostingClassifier

GradBC = GradientBoostingClassifier(random_state=0)
GradBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [36]:
pred_gradbc = GradBC.predict(xv_test)

In [37]:
GradBC.score(xv_test, y_test)

0.9954545454545455

In [38]:
print(classification_report(y_test, pred_gradbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5885
           1       0.99      1.00      1.00      5335

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



# TESTING THE PREDICTION MODELS

In [39]:
def output_lable(inp):
    if inp == 0:
        return "Fake News"
    elif inp == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testingNews = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LogReg = LogReg.predict(new_xv_test)
    pred_DecTree = DecTree.predict(new_xv_test)
    pred_GradBC = GradBC.predict(new_xv_test)

    return print("\n\nLogistics Regression Prediction: {} \nDecision Tree Classification Prediction: {} \nGradient Boosting Classifier Prediction: {}".format(output_lable(pred_LogReg[0]),
                                                                                                                                                              output_lable(pred_DecTree[0]), 
                                                                                                                                                              output_lable(pred_GradBC[0])))